In [42]:
import nltk # для разбиения текста на слова
from sty import fg, bg, ef, rs # для цветного вывода на экран
from sty import Style, RgbFg
import numpy as np # для работы с массивами
import re # для удаления знаков препинания
from nltk.tokenize import word_tokenize # текст на слова
from sklearn.cluster import KMeans # для кластеризации цветов
from math import sqrt # для расчета расстояния между цветами (и поиска ближайшего)
import pandas as pd # для работы с таблицей

Текст разбить на слова

В каждом слове получить цвет слова

Цвет слова получается сложением цветов его гласных + делением на кол-во гласных либо черный

1) выводим каждое слово своим цветом 

2) разбиваем текст на n кластеров, для каждого слова ищем ближайший цвет, выводим на экран найденным цветом

3) выводим таблицу частотности звуков (по формулам из презентации, они отражены в коде)

In [4]:
# Получение цвета слова
def colorWord(word):  
  word = list(word)
  wordColor = np.array([0, 0, 0])
  letterCount = 0
  for i in range(len(word)):
    if word[i] in alphabet:
     # print(word[i], alphabet[word[i]])
      letterCount += 1
      wordColor += alphabet[word[i]]      
  if letterCount > 0:
    wordColor = np.round(wordColor / letterCount).astype(int)
  return wordColor  

In [125]:
# Фукнция получения цветного слова 
def getColor(word, color):
  fg.color = Style(RgbFg(color[0], color[1], color[2]))
  wordColor = fg.color + word
  return wordColor


In [5]:
# Вывод слова в его цвете
def printColor(word, color):
  fg.color = Style(RgbFg(color[0], color[1], color[2]))
  wordColor = fg.color + word
  print(wordColor, end=' ', flush=True)

In [6]:
# Чтение текста из файла
def readTxt(filename):
  with open(filename) as f:
    lines = f.readlines()
  return lines

In [7]:
# Разбитие текста на строчки
def textWords(filename):
  lines = readTxt(filename)
  lineList = []
  for line in lines:
    line = re.sub('[\]\()[,;:\–\«\-!»?@#.$]', '', line)
    line = word_tokenize(line, language="russian")    
    lineList.append(line)    
  return lineList

In [8]:
# Вывод текста в цвете
def printText(wordList):
  colorArray = np.array([[0,0,0]])
  colorArray = np.delete(colorArray, 0, 0)
  for i in range(len(wordList)):
    wordList[i] = wordList[i].lower()
    color = colorWord(wordList[i])
    colorArray = np.append(colorArray, [color], axis=0)
    printColor(wordList[i], color)    
  return colorArray

Задаем rgb цвета для гласных


In [116]:
a = np.array([153, 0, 0])
ya = np.array([255, 0, 0])
o = np.array([255, 255, 204])
e = np.array([0, 153, 0])
i = np.array([0, 0, 153])
yi = np.array([24, 127, 171])
u = np.array([153, 0, 204])
yu = np.array([158, 190, 255])
any = np.array([0, 0, 0])

In [10]:
alphabet = {'а': a, 'е': e, 'ё': e, 'и': i, 'й': yi, 'о': o, 'у': u, 'ю': yu, 'я': ya, 'ы': any}

In [11]:
# Поиск ближайшего цвета
def closest_color(rgb, COLORS):
    r, g, b = rgb
    color_diffs = []
    for color in COLORS:
        cr, cg, cb = color
        color_diff = sqrt(abs(r - cr)**2 + abs(g - cg)**2 + abs(b - cb)**2)
        color_diffs.append((color_diff, color))
    return min(color_diffs)[1]

In [12]:
# Вывод текста в N цветах
def printTextCluster(wordList, COLORS):
  for i in range(len(wordList)):
    wordList[i] = wordList[i].lower()
    color = colorWord(wordList[i])
    color = closest_color(color, COLORS)
    printColor(wordList[i], color)

In [13]:
frequencyText = {'а': 0, 'е': 0, 'ё': 0, 'и': 0, 'й': 0, 'о': 0, 'у': 0, 'ю': 0, 'я': 0, 'ы': 0}
frequencyStandart = {'а': 0.116, 'е': 0.102, 'ё': 0, 'и': 0.077, 'й': 0, 'о': 0.126, 'у': 0.040, 'ю': 0, 'я': 0, 'ы': 0.024}

Посчитаем частоту гласных звуков

In [156]:
# Словарь с количеством букв в тексте
def letterFreq(lineList):
  all_freq = {}
  R = 0
  for line in lineList: 
    for word in line:
      for i in word:
       # i.lower()
        R += 1
        if i in all_freq: 
            all_freq[i] += 1
        else: 
            all_freq[i] = 1

  print('Всего звукобукв в тексте: ', R)
  freqText = {'а': 0, 'е': 0, 'ё': 0, 'и': 0, 'й': 0, 'о': 0, 'у': 0, 'ю': 0, 'я': 0, 'ы': 0}
  freqText = {k: freqText.get(k, 0) + all_freq.get(k, 0) for k in set(freqText)}
  print(freqText)
  df = pd.DataFrame(columns=['Звукобуква', 'Pi', 'Pin', 'S', 'Z', 'Цвет'])
  df['Звукобуква'] = ['О+0.5 Ё', 'А+Я', "Е+0.5 Ё", "И+0.5 Й", "У + Ю", "Ы"]
  df['Pin'] = [0.126, 0.116, 0.102, 0.077, 0.040, 0.024]
  df['S'] = (1 - df.Pin) * df.Pin / R
  df['S'] = round(df['S'].apply(np.sqrt), 3)

  freqText['о'] = round((0.5 * freqText['ё'] + freqText['о']) / R, 3)
  freqText['а'] = round((freqText['а'] + freqText['я']) / R, 3)
  freqText['е'] = round((freqText['е'] + 0.5 * freqText['ё']) / R, 3)
  freqText['и'] = round((freqText['и'] + 0.5 * freqText['й']) / R, 3)
  freqText['у'] = round((freqText['у'] + 0.5 * freqText['ю']) / R, 3)
  freqText['ы'] = round(freqText['ы'] / R, 3)
  df['Pi'] = [freqText['о'], freqText['а'], freqText['е'], freqText['и'], freqText['у'], freqText['ы']]
  df['Z'] = round((df['Pi'] - df['Pin']) / df['S'], 2)
  df['Цвет'] = ['бело-желтый', 'красный', 'зеленый', 'синий', 'фиолетовый', 'черный']
  print(df)
  return freqText

In [153]:
# Тест программы
# на вход название файла и кол-во кластеров (сколько сделать разных цветов)
def colorSemantic(filename, nClusters):  
  lineList = textWords(filename) 
  colorArray = np.array([[0,0,0]])
  colorArray = np.delete(colorArray, 0, 0)
  title =  ef.italic + 'Каждому слову свой цвет\n' + rs.italic
  print(title)  
  for line in lineList:     
    colorArray = np.append(colorArray, printText(line), axis=0)
    print()
  title =  fg.black + ef.italic + '\nВ тексте нашлось ' +str(len(colorArray))+' цветов\n' + rs.italic
  print(title) 
  kmeans = KMeans(n_clusters=nClusters, random_state=0).fit(colorArray)
  kmeans.cluster_centers_ = np.round(kmeans.cluster_centers_).astype(int)
  COLORS = kmeans.cluster_centers_  
  subtitle = fg.black + ef.italic + 'Текст в '+str(nClusters)+' цветах:\n' + rs.italic
  print(subtitle)
  for line in lineList:     
    printTextCluster(line, COLORS)
    print()
  subtitle = fg.black + ef.italic + '\nТаблица частотности:\n' + rs.italic
  print(subtitle)
  all_freq = letterFreq(lineList)  

In [165]:
filename = 'листопад.txt'
colorSemantic(filename, 3)

Каждому слову свой цвет

все разошлись на прощанье осталась 
оторопь желтой листвы за окном 
вот и осталась мне самая малость 
шороха осени в доме моем 
выпало лето холодной иголкой из онемелой руки тишины 
и запропало в потемках за полкой 
за штукатуркой мышиной стены 
если считаться начнем я не вправе 
даже на этот пожар за окном 
верно еще рассыпается гравий 
под осторожным ее каблуком 
там в заоконном тревожном покое 
вне моего бытия и жилья 
в желтом и синем и красном на что ей 
память моя что ей память моя 

В тексте нашлось 84 цветов

Текст в 3 цветах:

все разошлись на прощанье осталась 
оторопь желтой листвы за окном 
вот и осталась мне самая малость 
шороха осени в доме моем 
выпало лето холодной иголкой из онемелой руки тишины 
и запропало в потемках за полкой 
за штукатуркой мышиной стены 
если считаться начнем я не вправе 
даже на этот пожар за окном 
верно еще рассыпается гравий 
под осторожным ее каблуком 
там в заоконном тревожном покое 
вне моего бытия и жилья 
в желто